In [1]:
%matplotlib inline
import numpy as np
import sslearn
from sslearn import model_selection, wrapper
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample



Bad key "text.kerning_factor" on line 4 in
/home/jlgarridol/.miniconda3/envs/PhD/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import sklearn.datasets as data

In [3]:
X, y = data.load_iris(return_X_y=True)

In [4]:
resampling = resample(X, y, n_samples=int(X.shape[0]/3))

In [5]:
r1, r2 = resampling

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=float(1/3), random_state=0)

In [7]:
X_train.shape

(50, 4)

In [8]:
y_train

array([0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2,
       0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0,
       0, 0, 2, 1, 2, 0])

In [35]:
from sslearn.supervised.rotation import RotationForestClassifier

rt = RotationForestClassifier(base_estimator=DecisionTreeClassifier(random_state=10), n_estimators=10, random_state=5).fit(X_train, y_train)
rt.score(X_test, y_test)

0.95

Prueba de Validación cruzada

In [20]:
skf = model_selection.StratifiedKFoldSS(n_splits=10, random_state=0, shuffle=True)
scores = []
scores_ind = []
for X_, y_, _, unlabel in skf.split(X_train,y_train):
    st = wrapper.RelRasco(base_estimator=DecisionTreeClassifier(random_state=10), random_state=5, incremental=False)
    st.fit(X_,y_)
    scores.append(st.score(X_train[unlabel], y_train[unlabel]))
    scores_ind.append(st.score(X_test, y_test))
print("Precisión transductiva:",np.mean(scores))
print("Precisión inductiva:",np.mean(scores_ind))

Precisión transductiva: 0.8200000000000001
Precisión inductiva: 0.763


In [21]:
skf = model_selection.StratifiedKFoldSS(n_splits=10, random_state=0, shuffle=True)
scores = []
scores_ind = []
for X_, y_, _, unlabel in skf.split(X_train,y_train):
    st = wrapper.Rasco(base_estimator=DecisionTreeClassifier(random_state=10), random_state=5, incremental=False)
    st.fit(X_,y_)
    scores.append(st.score(X_train[unlabel], y_train[unlabel]))
    scores_ind.append(st.score(X_test, y_test))
print("Precisión transductiva:",np.mean(scores))
print("Precisión inductiva:",np.mean(scores_ind))

Precisión transductiva: 0.8911111111111112
Precisión inductiva: 0.834


In [22]:
skf = model_selection.StratifiedKFoldSS(n_splits=10, random_state=0, shuffle=True)
scores = []
scores_ind = []
for X_, y_, _, unlabel in skf.split(X_train,y_train):
    st = wrapper.RotRelRasco(base_estimator=DecisionTreeClassifier(random_state=10), random_state=5, incremental=False)
    st.fit(X_,y_)
    scores.append(st.score(X_train[unlabel], y_train[unlabel]))
    scores_ind.append(st.score(X_test, y_test))
print("Precisión transductiva:",np.mean(scores))
print("Precisión inductiva:",np.mean(scores_ind))

Precisión transductiva: 0.9066666666666666
Precisión inductiva: 0.8619999999999999


In [23]:
skf = model_selection.StratifiedKFoldSS(n_splits=10, random_state=0, shuffle=True)
scores = []
scores_ind = []
for X_, y_, _, unlabel in skf.split(X_train,y_train):
    st = wrapper.RotRelRasco(base_estimator=DecisionTreeClassifier(random_state=10), random_state=5, incremental=False, pre_rotation=True)
    st.fit(X_,y_)
    scores.append(st.score(X_train[unlabel], y_train[unlabel]))
    scores_ind.append(st.score(X_test, y_test))
print("Precisión transductiva:",np.mean(scores))
print("Precisión inductiva:",np.mean(scores_ind))

Precisión transductiva: 0.7733333333333333
Precisión inductiva: 0.724


In [36]:
skf = model_selection.StratifiedKFoldSS(n_splits=10, random_state=0, shuffle=True)
scores = []
scores_ind = []
for X_, y_, _, unlabel in skf.split(X_train,y_train):
    st = wrapper.SelfTraining(
        base_estimator=RotationForestClassifier(base_estimator=DecisionTreeClassifier(random_state=10),
                                                n_estimators=10, random_state=5), 
        max_iter=40)
    st.fit(X_,y_)
    scores.append(st.score(X_train[unlabel], y_train[unlabel]))
    scores_ind.append(st.score(X_test, y_test))
print("Precisión transductiva:",np.mean(scores))
print("Precisión inductiva:",np.mean(scores_ind))

Precisión transductiva: 0.9066666666666666
Precisión inductiva: 0.8619999999999999


Prueba creando un dataset artificial a partir de otro

In [15]:
X_, y_, true_label = model_selection.artificial_ssl_dataset(X_train, y_train, label_rate=0.10, random_state=0)
st = wrapper.CoForest(n_estimators=7, threshold=0.75)
score = st.fit(X_,y_).score(X_[y_ == -1], true_label)
score_ind = st.score(X_test, y_test)
print("Precisión transductiva:",score)
print("Precisión inductiva:",score_ind)

Precisión transductiva: 0.8222222222222222
Precisión inductiva: 0.82


In [16]:
X_, y_, true_label = model_selection.artificial_ssl_dataset(X_train, y_train, label_rate=0.05, random_state=0)
X_[y_==-1].shape, true_label.shape

((48, 4), (48,))

In [17]:
st.fit(X_,y_)
#.score(X_[y_ == -1], true_label)